In [4]:
import pandas as pd
import sys

# Data set preprocessing

In [76]:
# Import multiple latency values
p50 = pd.read_csv("Dataset/p50_latency_gcp.csv")
p95 = pd.read_csv("Dataset/p95_latency_gcp.csv")
p99 = pd.read_csv("Dataset/p99_latency_gcp.csv")
gcp_rps = pd.read_csv("Dataset/gcp_rps.csv")

#Concatenate multiple values
df = pd.concat([p50,p95,p99,gcp_rps], axis=1)

In [81]:
# Assign WEIGHTS
# Configure weights - The idea is to give a very low importance to those services that are not direclty connected
## Multiply *1 - Direct connection
## Multiply *0.3 - 2nd  connection ( Payment , email, redis and product catalog)
weight = 0.3

for col in df.columns:
        if 'redis' in col:
            df[col] = df[col].multiply(weight)
        if 'email' in col:
            df[col] = df[col].multiply(weight)
        if 'payment' in col:
            df[col] = df[col].multiply(weight)

#Send target to the end
s = df.pop('p95 deploy/frontend')
df = pd.concat([df, s], 1)

# Time is multiple times present. I need to keep just 1 sample for time since all of them will have the same value
df=df.loc[:, ~df.columns.duplicated(keep='first')]

#Convert time column to time format
df['Time'] = pd.to_datetime(df['Time'])
df.head()

/var/folders/hh/z9bqb5y14vs83vfz56nrhwtw0000gs/T/ipykernel_48420/1307003931.py:17: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df = pd.concat([df, s], 1)


,Time,p50 deploy/adservice,p50 deploy/cartservice,p50 deploy/checkoutservice,p50 deploy/currencyservice,p50 deploy/emailservice,p50 deploy/frontend,p50 deploy/loadgenerator,p50 deploy/paymentservice,p50 deploy/productcatalogservice,...,deploy/currencyservice,deploy/emailservice,deploy/frontend,deploy/loadgenerator,deploy/paymentservice,deploy/productcatalogservice,deploy/recommendationservice,deploy/redis-cart,deploy/shippingservice,p95 deploy/frontend
0,2021-10-07 09:18:00,3.166667,1.257143,23.333333,0.552147,0.01800,43.235294,0.75,0.016875,0.504373,...,0.2,0.0054,0.4,0.2,0.0054,0.2,0.2,0.0054,0.20002,131.000000
1,2021-10-07 09:18:05,2.500000,1.500000,10.000000,0.552147,0.02025,43.235294,0.75,0.016875,0.506309,...,0.2,0.0054,0.4,0.2,0.0054,0.2,0.2,0.0054,0.20002,131.000000
2,2021-10-07 09:18:10,2.500000,1.500000,10.000000,0.551136,0.02025,41.071429,0.75,0.027000,0.506309,...,0.2,0.0054,0.4,0.2,0.0054,0.2,0.2,0.0054,0.20000,99.027778
3,2021-10-07 09:18:15,2.194444,1.354839,25.000000,0.551136,0.02430,41.071429,0.75,0.027000,0.502907,...,0.2,0.0054,0.4,0.2,0.0054,0.2,0.2,0.0054,0.20000,99.027778
4,2021-10-07 09:18:20,2.194444,1.354839,25.000000,0.541985,0.02430,41.111111,0.75,0.023625,0.502907,...,0.2,0.0054,0.4,0.2,0.0054,0.2,0.2,0.0054,0.20000,240.000000
